In [1]:
import requests
import numpy as np
import pandas as pd
import altair as alt

In [17]:
alt.data_transformers.disable_max_rows()
alt.renderers.set_embed_options(format_locale="nl-NL", time_format_locale="nl-NL")
alt.themes.enable('latimes')

ThemeRegistry.enable('latimes')

In [42]:
dict_dfs = []
locations = ['IJSS', 'STEE', 'DOES', 'ZUTP', 'DEVE', 'OLST', 'WIJH', 'KATV', 'KAMP', 'KETD']
afstand = [878.5, 890.5, 903, 929.3, 945, 957, 965, 980.8, 994.5, 1001]
base_url = f"https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00%3A00Z"

In [43]:
for idx, _ in enumerate(locations):
    loc = locations[idx]
    dst = afstand[idx]
    loc_url = f"{base_url}&locationCode={loc}"
    print(loc_url)
    r = requests.get(loc_url).json()['results'][0]
    
    events = r['events']
    loc_name = r['location']['properties']['locationName']
    t_hoog = r['observationType']['thresholds'][-2]
    if t_hoog['name'] == 'hoog':
        t_hoog_level = t_hoog['level'] / 100
    else:
        print('error!') 
    df_loc = pd.DataFrame.from_records(events)
    df_loc['locatie'] = loc_name
    df_loc.value = df_loc.value / 100
    df_loc['waterstand_tov_hoog'] = df_loc.value - t_hoog_level
    df_loc['afstand'] = dst
    dict_dfs.append(df_loc)
df_locs = pd.concat(dict_dfs)
df_locs = df_locs.dropna()

https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00%3A00Z&locationCode=IJSS
https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00%3A00Z&locationCode=STEE
https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00%3A00Z&locationCode=DOES
https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00%3A00Z&locationCode=ZUTP
https://waterberichtgeving.rws.nl/wb/data/api/dd/2.0/timeseries?observationTypeId=WT&sourceName=S_1&thresholds=default&startTime=2023-12-22T15%3A00%3A00Z&endTime=2023-12-29T15%3A00

In [44]:
df_locs.to_json('series_tov_hoog.json', orient='records')
url_df = 'https://raw.githubusercontent.com/mattijn/dataset_hoogwater23/main/series_tov_hoog.json'

In [50]:
chart_line = alt.Chart(url_df).mark_line(tooltip=True).encode(
    x=alt.X('yearmonthdatehours(timeStamp):T').title(None), 
    y=alt.Y('max(waterstand_tov_hoog):Q').axis(title="waterstand t.o.v. drempelwaarde hoog"), 
    color=alt.Color('locatie:N').scale(
        domain=df_locs.locatie.unique().tolist(),
        range=['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', '#DDCC77' ,'#CC6677' , '#882255', '#AA4499', '#DDDDDD']
    )
)
chart_rule = alt.Chart().mark_rule().encode(y=alt.datum(0))
(chart_line + chart_rule)

alt.LayerChart(...)

In [51]:
chart_bar = alt.Chart(url_df).mark_line(clip=True).encode(
    x=alt.X('afstand:Q').axis(values=afstand, title='rivier kilometer').scale(domain=[np.min(afstand), np.max(afstand)]),
    y=alt.Y('waterstand_tov_hoog:Q').scale(domain=[-1, 0.20]).axis(title='waterstand t.o.v. drempelwaarde hoog'),
)
chart_point = alt.Chart(url_df).mark_point(tooltip=True, clip=True, filled=True, size=100, opacity=1).encode(
    x=alt.X('afstand:Q').axis(values=afstand, title='rivier kilometer'), 
    y=alt.Y('waterstand_tov_hoog:Q').scale(domain=[-1, 0.20]).axis(title='waterstand t.o.v. drempelwaarde hoog'),
    color=alt.Color('locatie:N').scale(
        domain=df_locs.locatie.unique().tolist(),
        range=['#332288', '#88CCEE', '#44AA99', '#117733', '#999933', '#DDCC77' ,'#CC6677' , '#882255', '#AA4499', '#DDDDDD']
    )
)

chart_rule = alt.Chart().mark_rule().encode(y=alt.datum(0))
(chart_bar + chart_point + chart_rule)

alt.LayerChart(...)

In [52]:
def timestamp(t):
  return pd.to_datetime(t).timestamp() * 1000

slider = alt.binding_range(
    step=10 * 60 * 1000, # 10 minutes in milliseconds
    min=timestamp(min(df_locs['timeStamp'])),
    max=timestamp(max(df_locs['timeStamp']))
)

select_date = alt.selection_point(
    fields=['timeStamp'],
    bind=slider,
    value=[{'timeStamp': timestamp(min(df_locs['timeStamp']))}],
    name='slider')

In [53]:
chart_all = alt.layer(
    chart_rule,
    chart_bar.add_params(select_date).transform_filter(
        "(year(datum.timeStamp) == year(slider.timeStamp[0])) && "
        "(month(datum.timeStamp) == month(slider.timeStamp[0])) && "
        "(date(datum.timeStamp) == date(slider.timeStamp[0])) && "
        "(hours(datum.timeStamp) == hours(slider.timeStamp[0])) && "
        "(minutes(datum.timeStamp) == minutes(slider.timeStamp[0]))"
    ),
    chart_point.add_params(select_date).transform_filter(
        "(year(datum.timeStamp) == year(slider.timeStamp[0])) && "
        "(month(datum.timeStamp) == month(slider.timeStamp[0])) && "
        "(date(datum.timeStamp) == date(slider.timeStamp[0])) && "
        "(hours(datum.timeStamp) == hours(slider.timeStamp[0])) && "
        "(minutes(datum.timeStamp) == minutes(slider.timeStamp[0]))"
    )
).properties(title=alt.Title(alt.expr(f'"situatie op "+date({select_date.name}["timeStamp"][0])+" december "+hours({select_date.name}["timeStamp"][0])+":"+minutes({select_date.name}["timeStamp"][0])')))
chart_all

alt.LayerChart(...)

In [54]:
print(chart_all.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.15.1.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "datasets": {
    "empty": [
      {}
    ]
  },
  "layer": [
    {
      "data": {
        "name": "empty"
      },
      "encoding": {
        "y": {
          "datum": 0
        }
      },
      "mark": {
        "type": "rule"
      }
    },
    {
      "data": {
        "url": "https://raw.githubusercontent.com/mattijn/dataset_hoogwater23/main/series_tov_hoog.json"
      },
      "encoding": {
        "x": {
          "axis": {
            "title": "rivier kilometer",
            "values": [
              878.5,
              890.5,
              903,
              929.3,
              945,
              957,
              965,
              980.8,
              994.5,
              1001
            ]
          },
          "field": "afstand",
          "scale": {
            "domain": [
              878.5,
   